In [1]:
import scenedetect
import numpy as np
import skvideo.datasets as datasets

In [2]:
filename = 'goldeneye.mp4'
#filename = datasets.bikes()

In [3]:
detector = scenedetect.detectors.ContentDetector()

In [4]:
scene_manager = scenedetect.SceneManager(detector = detector, downscale_factor=4)
fps, frames, processed = scenedetect.detect_scenes_file(filename, scene_manager)

[PySceneDetect] Parsing video goldeneye.mp4...
[PySceneDetect] Video Resolution / Framerate: 1280 x 544 / 23.976 FPS
[PySceneDetect] Subsampling Enabled (4x, Resolution = 320 x 136)
Verify that the above parameters are correct (especially framerate, use --force-fps to correct if required).


In [5]:
scenes = scene_manager.scene_list
scenes_msec = [(1e3 * x) / fps for x in scenes]
scenes_time = [scenedetect.timecodes.get_string(x) for x in scenes_msec]
print("Scenes (%d):" % len(scene_manager.scene_list), scene_manager.scene_list)
print("FPS: %f | Frames: %d | Processed: %d" % (fps, frames, processed))

Scenes (20): [90, 210, 259, 374, 650, 818, 876, 1019, 1055, 1099, 1174, 1226, 1260, 1281, 1334, 1365, 1697, 1871, 1916, 1966]
FPS: 23.976024 | Frames: 1980 | Processed: 1980


In [6]:
for i, frame, ms, time in zip(range(len(scenes)), scenes, scenes_msec, scenes_time):
    print("Scene %d at %s [Frame %d (%dms)]" % (i, time, frame, ms))

Scene 0 at 00:00:03.753 [Frame 90 (3753ms)]
Scene 1 at 00:00:08.758 [Frame 210 (8758ms)]
Scene 2 at 00:00:10.802 [Frame 259 (10802ms)]
Scene 3 at 00:00:15.598 [Frame 374 (15598ms)]
Scene 4 at 00:00:27.110 [Frame 650 (27110ms)]
Scene 5 at 00:00:34.117 [Frame 818 (34117ms)]
Scene 6 at 00:00:36.536 [Frame 876 (36536ms)]
Scene 7 at 00:00:42.500 [Frame 1019 (42500ms)]
Scene 8 at 00:00:44.002 [Frame 1055 (44002ms)]
Scene 9 at 00:00:45.837 [Frame 1099 (45837ms)]
Scene 10 at 00:00:48.965 [Frame 1174 (48965ms)]
Scene 11 at 00:00:51.134 [Frame 1226 (51134ms)]
Scene 12 at 00:00:52.552 [Frame 1260 (52552ms)]
Scene 13 at 00:00:53.428 [Frame 1281 (53428ms)]
Scene 14 at 00:00:55.638 [Frame 1334 (55638ms)]
Scene 15 at 00:00:56.931 [Frame 1365 (56931ms)]
Scene 16 at 00:01:10.779 [Frame 1697 (70779ms)]
Scene 17 at 00:01:18.036 [Frame 1871 (78036ms)]
Scene 18 at 00:01:19.913 [Frame 1916 (79913ms)]
Scene 19 at 00:01:21.998 [Frame 1966 (81998ms)]


In [15]:
import matplotlib.pyplot as plt
import imageio
from vispy import plot as vp, io as vio
import math


def draw_scenes_video(video, scenes, out='scenes.mp4'):
    reader: imageio.core.Reader = imageio.get_reader(video)
    meta = reader.get_meta_data()
    (width, height), frames, fps = meta['size'], meta['nframes'], meta['fps']
    graph = np.zeros((frames,))
    graph[scenes] = 1
    
    writer: imageio.core.Writer = imageio.get_writer(out, fps=fps)

    fig = vp.Fig(show=False, size=(width, height))
    fig1: vp.PlotWidget = fig[0, 0]
    fig1.plot((range(frames), graph), marker_size=0, color="blue", width=10, title="Detected Cuts", xlabel="Frame")
    fig1.camera.set_range(margin=0)
    seek_line: vp.LinePlot = fig1.plot(([0, 0], [0, 1]), width=10, edge_width=10)
    
    for i, frame in enumerate(reader):
        # Generate plot frame
        seek_line.set_data(([i, i], [0, 1]))
        plot_frame = fig.render()
        
        # Join video + plot
        out_frame = np.zeros((height, width*2, 3), dtype=np.uint8)
        out_frame[:, :width] = frame
        out_frame[:, width:] = plot_frame[:, :, :3]
        writer.append_data(out_frame)
        if (i % math.floor((fps * 2))) == 0:
            print("Progress: %d%%" % math.floor((i / frames) * 100))

    reader.close()
    writer.close()

In [16]:
draw_scenes_video(filename, scenes, "vispy_test.mp4")

Progress: 0%
Progress: 2%
Progress: 4%
Progress: 7%
Progress: 9%
Progress: 11%
Progress: 14%
Progress: 16%
Progress: 18%
Progress: 21%
Progress: 23%
Progress: 26%
Progress: 28%
Progress: 30%
Progress: 33%
Progress: 35%
Progress: 37%
Progress: 40%
Progress: 42%
Progress: 45%
Progress: 47%
Progress: 49%
Progress: 52%
Progress: 54%
Progress: 56%
Progress: 59%
Progress: 61%
Progress: 64%
Progress: 66%
Progress: 68%
Progress: 71%
Progress: 73%
Progress: 75%
Progress: 78%
Progress: 80%
Progress: 83%
Progress: 85%
Progress: 87%
Progress: 90%
Progress: 92%
Progress: 94%
Progress: 97%
Progress: 99%


/home/jcosta/.conda/envs/dissertation/lib/python3.6/site-packages/imageio/core/format.py:392: UserWarning: Could not read last frame of /home/jcosta/Documents/Workspace/scene-detection/goldeneye.mp4.
  warn('Could not read last frame of %s.' % uri)
